# 0.0 Data preparation

# Preparation

Imports libraries

Loads data

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline

print("0.0 data preparation")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.transforms as tr
import seaborn as sns
import re
import json
import datetime
import random
import os

from random import randint
from ipywidgets import FloatProgress,IntProgress,IntText,Text,interact,interactive,IntSlider,FloatSlider
from IPython.display import display
from itertools import chain
from scipy.stats import ttest_ind

In [ ]:
pd.__version__

In [ ]:
np.__version__

### common variables

In [ ]:
# questions & columns
indexColumn = 'index'
emailQuestion = 'Email'
timestampQuestion = 'Timestamp'
idQuestion = 'ID'
nameQuestion = 'Name'
gameQuestion = 'Which game have you just played?'
remarksQuestion = '12. Write here your remarks about the game: which feature was missing, what failed or glitched, what was great:'
questionCount = 12
firstLikertQuestionIndex = 0
lastLikertQuestionIndex = 11
firstSUSQuestionIndex = 0
lastSUSQuestionIndex = 10
questionArrayInt = [i+1 for i in range(questionCount)]
questionArrayStr = sorted(["Q" + "{0:0=2d}".format(i) for i in questionArrayInt])
#questionArrayStr

In [ ]:
def tryCreateFolder(path, displayMessage = False):
    try:  
        os.mkdir(path)
    except OSError:  
        if displayMessage:
            print ("Creation of the directory %s failed" % path)
    else:  
        if displayMessage:
            print ("Successfully created the directory %s " % path)

In [ ]:
# define the name of the directory to be created
graphsSavePathStem = "../Graphs"
tryCreateFolder(graphsSavePathStem)

In [ ]:
suffixSUSNormalized = "SUSNormalized"

In [ ]:
def generateTestPath(number):
    return dataFolderPath + testStem + str(number) + csvSuffix

In [ ]:
dataFolderPath = "../Data/"
unanonymizedDataFolderPath = dataFolderPath + "UnanonymizedData/"
csvSuffix = '.csv'
rawSuffix = "_event_raw"
rawAnonymizedSuffix = "_event_raw_unanonymized"
dataStem = "_data"
SUSNormalizedSuffix = "_SUSNormalized"

testStem = "test"

# Miranda House data
dateStemMirandaHouse                       = "2019-06-03"
unanonymizedDataFilesNamesStemMirandaHouse = dateStemMirandaHouse + rawAnonymizedSuffix
dataFilesNamesStemMirandaHouse             = dateStemMirandaHouse + rawSuffix
unanonymizedRaw20190603PathMirandaHouse    = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemMirandaHouse            + csvSuffix
raw20190603PathMirandaHouse                = dataFolderPath             + dataFilesNamesStemMirandaHouse                        + csvSuffix
data20190603Path                           = dataFolderPath             + dateStemMirandaHouse + dataStem                       + csvSuffix
data20190603SUSNormalizedPath              = dataFolderPath             + dateStemMirandaHouse + dataStem + SUSNormalizedSuffix + csvSuffix

# Cité des Sciences data
dateStemCiteDesSciences                       = "2019-07-03"
unanonymizedDataFilesNamesStemCiteDesSciences = dateStemCiteDesSciences + rawAnonymizedSuffix
dataFilesNamesStemCiteDesSciences             = dateStemCiteDesSciences + rawSuffix
unanonymizedRaw20190703PathCiteDesSciences    = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemCiteDesSciences            + csvSuffix
raw20190703PathCiteDesSciences                = dataFolderPath             + dataFilesNamesStemCiteDesSciences                        + csvSuffix
data20190703Path                              = dataFolderPath             + dateStemCiteDesSciences + dataStem                       + csvSuffix
data20190703SUSNormalizedPath                 = dataFolderPath             + dateStemCiteDesSciences + dataStem + SUSNormalizedSuffix + csvSuffix

# Miranda House 2 data
dateStemMirandaHouse2                      = "2019-08-02"
unanonymizedDataFilesNamesStemMirandaHouse2= dateStemMirandaHouse2 + rawAnonymizedSuffix
dataFilesNamesStemMirandaHouse2            = dateStemMirandaHouse2 + rawSuffix
unanonymizedRaw20190802PathMirandaHouse2   = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemMirandaHouse2            + csvSuffix
raw20190802PathMirandaHouse2               = dataFolderPath             + dataFilesNamesStemMirandaHouse2                        + csvSuffix
data20190802Path                           = dataFolderPath             + dateStemMirandaHouse2 + dataStem                       + csvSuffix
data20190802SUSNormalizedPath              = dataFolderPath             + dateStemMirandaHouse2 + dataStem + SUSNormalizedSuffix + csvSuffix

# Kigali data
dateStemKigali                       = "2019-08-28"
unanonymizedDataFilesNamesStemKigali = dateStemKigali + rawAnonymizedSuffix
dataFilesNamesStemKigali             = dateStemKigali + rawSuffix
unanonymizedRaw20190828PathKigali    = unanonymizedDataFolderPath + unanonymizedDataFilesNamesStemKigali            + csvSuffix
raw20190828PathKigali                = dataFolderPath             + dataFilesNamesStemKigali                        + csvSuffix
data20190828Path                     = dataFolderPath             + dateStemKigali + dataStem                       + csvSuffix
data20190828SUSNormalizedPath        = dataFolderPath             + dateStemKigali + dataStem + SUSNormalizedSuffix + csvSuffix


# data anonymization

In [ ]:
# Warning: works in-place
def fixData(dataToFix):
    questionsInvolved = [gameQuestion]
    wrongSpelling = "Fungal Invasion"
    correctSpelling = "Fungal Invaders"
    for answer in dataToFix.index:
        for question in questionsInvolved:
            if dataToFix.loc[answer, question] == wrongSpelling:
                dataToFix.loc[answer, question] = correctSpelling
    return dataToFix
        

#testData = raw20190603.copy()
#fixData(testData)
#testData.loc[:, questionsInvolved]        

In [ ]:
def fixRawData(rawPath):
    raw = pd.read_csv(rawPath, index_col=indexColumn, dtype=str)
    fixData(raw)
    raw.to_csv(rawPath, encoding='utf-8')
    
#fixRawData(raw20190603PathMirandaHouse)
#fixRawData(raw20190703PathCiteDesSciences)

In [ ]:
# returns anonymized data: replaces Name by identifier and removes the Email column
# reads from the unanomizedresult to the anonymizedPath
def preprocessAnonymize(unanonymizedPath, anonymizedPath):
    
    # TODO filter out tests by our team members and organizers
    
    unanonymizedData = pd.read_csv(unanonymizedPath,dtype=str)

    unanonymizedData = unanonymizedData.drop(axis=1, columns=[emailQuestion])

    nameSet = set(unanonymizedData[nameQuestion])
    nameSeries = pd.Series(list(nameSet))
    for answerIndex in unanonymizedData.index:
        name = unanonymizedData.loc[answerIndex, nameQuestion]
        unanonymizedData.loc[answerIndex, nameQuestion] = nameSeries.index[nameSeries == name][0]
    unanonymizedData.rename(columns={nameQuestion:idQuestion}, inplace=True)

    fixData(unanonymizedData)
        
    if indexColumn in unanonymizedData.columns:
        unanonymizedData = unanonymizedData.set_index(indexColumn)
        
    print("writing to " + anonymizedPath)
    unanonymizedData.to_csv(anonymizedPath, encoding='utf-8')
    
    return unanonymizedData

In [ ]:
# Python adds an artefact index column with indices 0,..,n _by default_ when reading then writing a csv without using the argument index_col=.
# This function fixes the consequences of forgetting to use this argument.
def getRidOfArtefactFirstColumn(rawPath):
    raw = pd.read_csv(rawPath,dtype=str)
    raw = raw.drop(axis=1, columns=raw.columns[0])
    raw = raw.set_index('Timestamp')
    raw.to_csv(rawPath, encoding='utf-8')

In [ ]:
doAnonymize = False

if doAnonymize:
    raw20190603 = preprocessAnonymize(\
        unanonymizedRaw20190603PathMirandaHouse,\
        raw20190603PathMirandaHouse)
    
if doAnonymize:
    raw20190703 = preprocessAnonymize(\
        unanonymizedRaw20190703PathCiteDesSciences,\
        raw20190703PathCiteDesSciences)
    
if doAnonymize:
    raw20190802 = preprocessAnonymize(\
        unanonymizedRaw20190802PathMirandaHouse2,\
        raw20190802PathMirandaHouse2)

if doAnonymize:
    # first merge the three separate data files
    unanonymizedPath1 = "../Data/UnanonymizedData/2019-08-28_website_Game-Questionnaire-Game1.csv"
    unanonymizedPath2 = "../Data/UnanonymizedData/2019-08-28_website_Game-Questionnaire-Game2.csv"
    unanonymizedPath3 = "../Data/UnanonymizedData/2019-08-28_website_Game-Questionnaire-Game3.csv"

    unanonymized1 = pd.read_csv(unanonymizedPath1,dtype=str)
    unanonymized2 = pd.read_csv(unanonymizedPath2,dtype=str)
    unanonymized3 = pd.read_csv(unanonymizedPath3,dtype=str)

    concatenated = pd.concat([unanonymized1, unanonymized2, unanonymized3])

    assert len(unanonymized1) + len(unanonymized2) + len(unanonymized3) == len(concatenated)

    concatenated.index = range(len(concatenated.index))
    concatenated.index.name = indexColumn
    concatenated.to_csv(unanonymizedRaw20190828PathKigali, encoding='utf-8')
    
    raw20190828 = preprocessAnonymize(\
        unanonymizedRaw20190828PathKigali,\
        raw20190828PathKigali)

# Functions

## Loading

In [ ]:
# Miranda House
try:
    raw20190603  = pd.read_csv(raw20190603PathMirandaHouse, index_col=indexColumn, dtype=str)
except:
    print("Miranda House data read failed: processing raw data...")
    raw20190603 = preprocessAnonymize(unanonymizedRaw20190603PathMirandaHouse, raw20190603PathMirandaHouse)
    raw20190603 = pd.read_csv(raw20190603PathMirandaHouse, index_col=indexColumn, dtype=str)

In [ ]:
# Cite des Sciences
try:
    raw20190703  = pd.read_csv(raw20190703PathCiteDesSciences, index_col=indexColumn, dtype=str)
except:
    print("Cite des Sciences data read failed: processing raw data...")
    raw20190703 = preprocessAnonymize(unanonymizedRaw20190703PathCiteDesSciences, raw20190703PathCiteDesSciences)
    raw20190703 = pd.read_csv(raw20190703PathCiteDesSciences, index_col=indexColumn, dtype=str)

In [ ]:
# Miranda House 2
try:
    raw20190802  = pd.read_csv(raw20190802PathMirandaHouse2, index_col=indexColumn, dtype=str)
except:
    print("Miranda House 2 data read failed: processing raw data...")
    raw20190802 = preprocessAnonymize(unanonymizedRaw20190802PathMirandaHouse2, raw20190802PathMirandaHouse2)
    raw20190802 = pd.read_csv(raw20190802PathMirandaHouse2, index_col=indexColumn, dtype=str)

In [ ]:
# Kigali
try:
    raw20190828  = pd.read_csv(raw20190828PathKigali, index_col=indexColumn, dtype=str)
except:
    print("Kigali data read failed: processing raw data...")
    raw20190828 = preprocessAnonymize(unanonymizedRaw20190828PathKigali, raw20190828PathKigali)
    raw20190828 = pd.read_csv(raw20190828PathKigali, index_col=indexColumn, dtype=str)

In [ ]:
assert ((raw20190603.columns == raw20190703.columns).all()), \
("column mismatch: data from different experiments must have the same columns")

## additional treatment: column name extraction
Game question
short, indexed, Likert, SUS questions

In [ ]:
firstQuestionIndex = len(raw20190603.columns)-len(questionArrayStr)

In [ ]:
shortQuestions = pd.Series(index=questionArrayStr, data=raw20190603.columns[firstQuestionIndex:])
shortQuestions

In [ ]:
indexedQuestions = pd.Series(index=range(1,13), data=raw20190603.columns[firstQuestionIndex:])
indexedQuestions

In [ ]:
indexedLikertQuestions = indexedQuestions[firstLikertQuestionIndex:lastLikertQuestionIndex]
indexedLikertQuestions

In [ ]:
positiveLikertQuestions = indexedLikertQuestions.copy()
positiveLikertQuestions = positiveLikertQuestions[positiveLikertQuestions.index % 2 == 1]
positiveLikertQuestions

In [ ]:
negativeLikertQuestions = indexedLikertQuestions.copy()
negativeLikertQuestions = negativeLikertQuestions[negativeLikertQuestions.index % 2 == 0]
negativeLikertQuestions

In [ ]:
shortLikertQuestions = shortQuestions[firstLikertQuestionIndex:lastLikertQuestionIndex]
shortLikertQuestions

In [ ]:
indexedSUSQuestions = indexedQuestions[firstSUSQuestionIndex:lastSUSQuestionIndex]
indexedSUSQuestions

In [ ]:
positiveSUSQuestions = indexedSUSQuestions.copy()
positiveSUSQuestions = positiveSUSQuestions[positiveSUSQuestions.index % 2 == 1]
positiveSUSQuestions

In [ ]:
negativeSUSQuestions = indexedSUSQuestions.copy()
negativeSUSQuestions = negativeSUSQuestions[negativeSUSQuestions.index % 2 == 0]
negativeSUSQuestions

In [ ]:
shortSUSQuestions = shortQuestions[firstSUSQuestionIndex:lastSUSQuestionIndex]
shortSUSQuestions

## 1-word question descriptions

In [ ]:
shortDescQuestions = pd.Series(index=raw20190603.columns[firstQuestionIndex:], data=[
     "frequency",				#01. I think that I would like to play this game frequently.',
     "complexity",				#02. I found the game unnecessarily complex.',
     "ease",					#03. I thought the game was easy to play.',
     "need for support",		#04. I think that I would need the support of a technical person to be able to play this game.',    
     "integration",				#05. I found the various functions in this game were well integrated.',
     "consistency",				#06. I thought there was too much inconsistency in this game.',
     "others' learning",		#07. I would imagine that most people would learn to play this game very quickly.',
     "convenience",				#08. I found the game very cumbersome to play.',
	 "confidence",				#09. I felt very confident using the game.',
	 "my learning",				#10. I needed to learn a lot of things before I could get going with this game.',
	 "recommendation",			#11. I would recommend this game to a friend.',
	 "remarks",					#12. Write here your remarks about the game: which feature was missing, what failed or glitched, what was great:',
	 ])
shortDescQuestions

In [ ]:
shortDescQuestions = pd.Series(index=raw20190603.columns[firstQuestionIndex:], data=[
     "replay appeal",			#01. I think that I would like to play this game frequently.',
     "simplicity",				#02. I found the game unnecessarily complex.',
     "ease",					#03. I thought the game was easy to play.',
     "autonomy",		        #04. I think that I would need the support of a technical person to be able to play this game.',    
     "integration",				#05. I found the various functions in this game were well integrated.',
     "consistency",				#06. I thought there was too much inconsistency in this game.',
     "learnable by others",		#07. I would imagine that most people would learn to play this game very quickly.',
     "convenience",				#08. I found the game very cumbersome to play.', maniability
	 "confidence",				#09. I felt very confident using the game.',
	 "learnable by self",	    #10. I needed to learn a lot of things before I could get going with this game.',
	 "recommendation",			#11. I would recommend this game to a friend.',
	 "remarks",					#12. Write here your remarks about the game: which feature was missing, what failed or glitched, what was great:',
	 ])
shortDescQuestions

## game names

In [ ]:
games = raw20190603[gameQuestion].unique()
games

In [ ]:
gameDrBugTitle = 'Dr Bug: Microbe Mayhem'
gameSuperbugsTitle = 'Superbugs: the Game'
gameFungalTitle = 'Fungal Invaders'

assert (set(games)==set((gameDrBugTitle, gameSuperbugsTitle, gameFungalTitle))), ("Wrong list of games")

In [ ]:
identityGameNames = pd.Series(index=[gameDrBugTitle, gameSuperbugsTitle, gameFungalTitle],
                           data=[gameDrBugTitle, gameSuperbugsTitle, gameFungalTitle])
shortGameNames = pd.Series(index=identityGameNames.index,
                           data=['Dr Bug', 'Superbugs', 'Fungal Invaders'])
shortGameNames

In [ ]:
def getShortGameTitle(longGameName):
    return shortGameNames.get(longGameName, "unknown")

### Pandas & Python syntax tests
Based on https://stackoverflow.com/questions/38309729/count-unique-values-with-pandas-per-groups/38309807

In [ ]:
if False:
    ''.join(['a','b','c'])

In [ ]:
if False:

    df = pd.DataFrame(index = range(9), columns=range(2))

    df.loc[0,0]=0
    df.loc[1,0]=1
    df.loc[2,0]=1
    df.loc[3,0]=2
    df.loc[4,0]=2
    df.loc[5,0]=2
    df.loc[6,0]=3
    df.loc[7,0]=3
    df.loc[8,0]=3

    df.loc[0,1]=0
    df.loc[1,1]=0
    df.loc[2,1]=1
    df.loc[3,1]=0
    df.loc[4,1]=1
    df.loc[5,1]=2
    df.loc[6,1]=0
    df.loc[7,1]=1
    df.loc[8,1]=1

    print(df)
    print()
    print(df.groupby(by=0, as_index=False).agg({1: pd.Series.nunique}))
    print()
    print(df.groupby(0).count())
    print()
    print(df.groupby(0).nunique())
    print()
    print(df.nunique())

In [ ]:
if False:
    testtotal = 0
    for x in range(4):
        print("x=" + str(x))
        if x % 2 == 0:
            for y in range(4):
                print("\ty=" + str(y))
                if y == 2:
                    print("\t\ty == 2")
                    break
                testtotal = testtotal + 1
                print("\t\telse: testtotal="+str(testtotal))
    testtotal

## advanced treatment: data refinement
Scores are stored as strings and must be converted to integers and NaNs for missing values

### Removing the players who haven't played all of the games

In [ ]:
#raw20190603.loc[:,[idQuestion,gameQuestion]].groupby(idQuestion).count()

In [ ]:
# should be the way to go
if False:
    groupByAgg = raw20190603.loc[:,[idQuestion,gameQuestion]].groupby(by=idQuestion, as_index=False).agg({gameQuestion: pd.Series.nunique})
    groupByAgg.to_csv(
        dataFolderPath + "groupByAgg" + csvSuffix,
        encoding='utf-8')

In [ ]:
# 
if False:
    groupByBothCount = raw20190603.loc[:,[idQuestion,gameQuestion]].groupby([idQuestion,gameQuestion]).count()
    groupByBothCount.to_csv(
        dataFolderPath + "groupByBothCount" + csvSuffix,
        encoding='utf-8')

In [ ]:
if False:
    groupByIdCount = raw20190603.loc[:,[idQuestion,gameQuestion]].groupby(idQuestion).count()
    groupByIdCount.to_csv(
        dataFolderPath + "groupByIdCount" + csvSuffix,
        encoding='utf-8')

In [ ]:
if False:
    playedAllGamesIDs = groupByAgg[groupByAgg[gameQuestion] == 3][idQuestion].values

In [ ]:
def getPlayedAllGamesIDs(anonymizedData):
    groupByAgg = anonymizedData.loc[:,[idQuestion,gameQuestion]].groupby(by=idQuestion, as_index=False).agg({gameQuestion: pd.Series.nunique})
    return groupByAgg[groupByAgg[gameQuestion] == 3][idQuestion].values

## Refining: filtering and computing function

In [ ]:
# the "break" instruction will save some time but a participant who played twice will still appear twice in the respondent list
def getNumericalData(anonymizedData):
    numericalData = anonymizedData.copy()
    idsToDrop = set()
    playedAllGamesIDs = getPlayedAllGamesIDs(anonymizedData)

    for respondent in numericalData.index:
        if numericalData.loc[respondent, idQuestion] in playedAllGamesIDs:
            for question in indexedLikertQuestions:
                try:
                    numericalData.loc[respondent, question] = int(numericalData.loc[respondent, question])
                except:
                    # option 1: use NaNs
                    # numericalData.loc[respondent, question] = np.nan
                    # option 2: exclude entry; make sure all data from this person are excluded for the sake of consistency.
                    idsToDrop.add(numericalData.loc[respondent, idQuestion])
                    break
        else:
            # print("id " + str(numericalData.loc[respondent, idQuestion]) + " did not play all the games")
            idsToDrop.add(numericalData.loc[respondent, idQuestion])
    
    # exclude participants who didn't answer some questions or didn't play some games
    numericalData.drop(
                        axis=0,
                        inplace=True,
                        index=numericalData[numericalData[idQuestion].isin(idsToDrop)].index)
    
    # exclude participants who didn't play all the games
    # TODO


    # reindex
    numericalData.index = range(len(numericalData.index))
    numericalData.index.name = indexColumn
    
    return numericalData

In [ ]:
def getNormalizedNumericalData(numericalData):
    # transforms the agreement scores (1-5 Likert scale) into a 0-4 mark with 0 = bad and 4 = great
    normalizedNumericalData = numericalData.copy()
    for respondent in normalizedNumericalData.index:
        for question in indexedLikertQuestions:
            answer = normalizedNumericalData.loc[respondent, question]
            if pd.notna(answer):
                if question in negativeLikertQuestions.values:            
                    normalizedNumericalData.loc[respondent, question] = 5-answer
                else:
                    normalizedNumericalData.loc[respondent, question] = answer-1
                    
    return normalizedNumericalData

In [ ]:
dtypeDict = dict()
for question in indexedLikertQuestions:
    dtypeDict[question] = "int"
dtypeDict

In [ ]:
# Miranda House data

try:
    data20190603               = pd.read_csv(data20190603Path, index_col=indexColumn, dtype=dtypeDict)
    data20190603SUSNormalized  = pd.read_csv(data20190603SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)
except:
    data20190603 = getNumericalData(raw20190603)
    data20190603SUSNormalized = getNormalizedNumericalData(data20190603)
    
    data20190603.to_csv(data20190603Path, encoding='utf-8')
    data20190603SUSNormalized.to_csv(data20190603SUSNormalizedPath, encoding='utf-8')
    
    data20190603               = pd.read_csv(data20190603Path, index_col=indexColumn, dtype=dtypeDict)
    data20190603SUSNormalized  = pd.read_csv(data20190603SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)


In [ ]:
# Cite des Sciences data

try:
    data20190703               = pd.read_csv(data20190703Path, index_col=indexColumn, dtype=dtypeDict)
    data20190703SUSNormalized  = pd.read_csv(data20190703SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)
except:
    data20190703 = getNumericalData(raw20190703)
    data20190703SUSNormalized = getNormalizedNumericalData(data20190703)
    
    data20190703.to_csv(data20190703Path, encoding='utf-8')
    data20190703SUSNormalized.to_csv(data20190703SUSNormalizedPath, encoding='utf-8')
    
    data20190703               = pd.read_csv(data20190703Path, index_col=indexColumn, dtype=dtypeDict)
    data20190703SUSNormalized  = pd.read_csv(data20190703SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)

In [ ]:
# Miranda House 2 data

try:
    data20190802               = pd.read_csv(data20190802Path, index_col=indexColumn, dtype=dtypeDict)
    data20190802SUSNormalized  = pd.read_csv(data20190802SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)
except:
    data20190802 = getNumericalData(raw20190802)
    data20190802SUSNormalized = getNormalizedNumericalData(data20190802)
    
    data20190802.to_csv(data20190802Path, encoding='utf-8')
    data20190802SUSNormalized.to_csv(data20190802SUSNormalizedPath, encoding='utf-8')
    
    data20190802               = pd.read_csv(data20190802Path, index_col=indexColumn, dtype=dtypeDict)
    data20190802SUSNormalized  = pd.read_csv(data20190802SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)

In [ ]:
# Kigali data

try:
    data20190828               = pd.read_csv(data20190828Path, index_col=indexColumn, dtype=dtypeDict)
    data20190828SUSNormalized  = pd.read_csv(data20190828SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)
except:
    data20190828 = getNumericalData(raw20190828)
    data20190828SUSNormalized = getNormalizedNumericalData(data20190828)
    
    data20190828.to_csv(data20190828Path, encoding='utf-8')
    data20190828SUSNormalized.to_csv(data20190828SUSNormalizedPath, encoding='utf-8')
    
    data20190828               = pd.read_csv(data20190828Path, index_col=indexColumn, dtype=dtypeDict)
    data20190828SUSNormalized  = pd.read_csv(data20190828SUSNormalizedPath, index_col=indexColumn, dtype=dtypeDict)

In [ ]:
datasets = dict()

datasets["data20190603"]              = data20190603
datasets["data20190603SUSNormalized"] = data20190603SUSNormalized
datasets["data20190703"]              = data20190703
datasets["data20190703SUSNormalized"] = data20190703SUSNormalized
datasets["data20190802"]              = data20190802
datasets["data20190802SUSNormalized"] = data20190802SUSNormalized
datasets["data20190828"]              = data20190828
datasets["data20190828SUSNormalized"] = data20190828SUSNormalized

In [ ]:
def selectDataset(datasetName):
    #_dataName = "data20190603"
    _dataName = datasetName
    assert (_dataName in datasets), ("Not found in datasets: '" + _dataName + "'")
    inputData=datasets[_dataName]

    _dataNameSUSNormalized = _dataName + suffixSUSNormalized
    assert (_dataName in datasets), ("Not found in datasets: '" + _dataNameSUSNormalized + "'")
    inputDataSUSNormalized=datasets[_dataNameSUSNormalized]
    
    return _dataName, inputData, _dataNameSUSNormalized, inputDataSUSNormalized

In [ ]:
# digital5StepDescriptions = range(minLikertValue, maxLikertValue+1)
# mixed5StepDecriptions = ['Strongly agree', '2', '3', '4', 'Strongly disagree']
# likert5StepDescriptions = ['Strongly agree', 'Slightly agree', 'Neutral', 'Slightly disagree', 'Strongly disagree']
likert5StepDescriptions = ['Strongly agree', 'Agree', 'Neutral', 'Disagree', 'Strongly disagree']

In [ ]:
def getMaxAnswers(dataname):
    return max([len(datasets[_dataName][datasets[_dataName][gameQuestion]==gameTitle]) for gameTitle in games])

In [ ]:
np.unique(data20190603[idQuestion].values)

## Checks on the number of participants who played 3 games and provided complete answer forms

In [ ]:
#for (_dataset, _datasetName) in zip([raw20190603, raw20190703, raw20190802, raw20190828], ["raw20190603", "raw20190703", "raw20190802", "raw20190828"]):
#    print(_datasetName + "\n" + str(_dataset[gameQuestion].value_counts()) + "\n\n")
#    _dataset.loc[:,[idQuestion, gameQuestion]].sort_values([idQuestion, gameQuestion]).to_csv(dataFolderPath + "IDs_&_Game_" + _datasetName + csvSuffix)

In [ ]:
# check that experimenters were excluded in filtering!


# raw20190603
# played 3 games:
# played3 = [0, 1, 10, 12, 13, 14, 15, 16, 17, 18, 19, 2, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 4, 5, 6, 7, 8, 9]
# did not:
# playedLess3 = [11, 20]

# raw20190703
# played 3 games:
# played3 = [15, 20, 6, 8]
# did not:
# playedLess3 = [x for x in list(range(25)) if x not in played3]

# raw20190828
# played 3 games:
# played3 = [3, 30, 31, 34, 4, 5, 8]
# did not:
# playedLess3 = [x for x in list(range(58)) if x not in played3]

# raw20190802
# played 3 games:
# played3 = ?
# did not:
# playedLess3 = ?


In [ ]:
# comparison with data files ID lists
for (_dataset, _datasetName) in zip([data20190603, data20190703, data20190802, data20190828], ["data20190603", "data20190703", "data20190802", "data20190828"]):
    print(_datasetName)
    print(str(list(np.unique(_dataset[idQuestion].values))))
    print()

In [ ]:
# diff between raw20190603's played 3 games and data20190603's IDs
if False:
    played3 = [0, 1, 10, 12, 13, 14, 15, 16, 17, 18, 19, 2, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 4, 5, 6, 7, 8, 9]
    indata = [0, 2, 3, 5, 6, 7, 8, 10, 13, 14, 15, 16, 17, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33]
    print([x for x in played3 if x not in indata])
    print([x for x in indata if x not in played3])
# [1, 12, 18, 19, 22, 27, 4, 9]
# []
# remains to be accounted for: 

In [ ]:
allFaulty = []
if True:
    for column in indexedLikertQuestions:
        faultyIDs = raw20190603.loc[raw20190603[column].isna()][idQuestion]
        if len(faultyIDs) > 0:
            #print("  allFaulty: " + str(list(allFaulty)))
            #print("+ faultyIDs: " + str(list(faultyIDs.values)))
            allFaulty = list(np.unique(allFaulty + list(faultyIDs.values)))
            #print("= allFaulty: " + str(list(allFaulty)))
            #print()
allFaulty